## CBoW 모델 훈련

이 노트북은 [AI for Beginners Curriculum](http://aka.ms/ai-beginners)의 일부입니다.

이 예제에서는 CBoW 언어 모델을 훈련하여 자체 Word2Vec 임베딩 공간을 얻는 방법을 살펴보겠습니다. 텍스트 소스로 AG News 데이터셋을 사용할 것입니다.


In [ ]:
import torch
import torchtext
import os
import collections
import builtins
import random
import numpy as np

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

먼저 데이터셋을 로드하고 토크나이저와 어휘를 정의합시다. 계산을 약간 제한하기 위해 `vocab_size`를 5000으로 설정하겠습니다.


In [ ]:
def load_dataset(ngrams = 1, min_freq = 1, vocab_size = 5000 , lines_cnt = 500):
    tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
    print("Loading dataset...")
    test_dataset, train_dataset  = torchtext.datasets.AG_NEWS(root='./data')
    train_dataset = list(train_dataset)
    test_dataset = list(test_dataset)
    classes = ['World', 'Sports', 'Business', 'Sci/Tech']
    print('Building vocab...')
    counter = collections.Counter()
    for i, (_, line) in enumerate(train_dataset):
        counter.update(torchtext.data.utils.ngrams_iterator(tokenizer(line),ngrams=ngrams))
        if i == lines_cnt:
            break
    vocab = torchtext.vocab.Vocab(collections.Counter(dict(counter.most_common(vocab_size))), min_freq=min_freq)
    return train_dataset, test_dataset, classes, vocab, tokenizer

In [ ]:
train_dataset, test_dataset, _, vocab, tokenizer = load_dataset()

Loading dataset...
Building vocab...


In [ ]:
def encode(x, vocabulary, tokenizer = tokenizer):
    return [vocabulary[s] for s in tokenizer(x)]

## CBoW 모델

CBoW는 $2N$개의 주변 단어를 기반으로 단어를 예측하는 방법을 학습합니다. 예를 들어, $N=1$일 때, 문장 *I like to train networks*에서 다음과 같은 쌍을 얻을 수 있습니다: (like,I), (I, like), (to, like), (like,to), (train,to), (to, train), (networks, train), (train,networks). 여기서 첫 번째 단어는 입력으로 사용되는 주변 단어이고, 두 번째 단어는 우리가 예측하려는 단어입니다.

다음 단어를 예측하는 네트워크를 구축하려면, 주변 단어를 입력으로 제공하고 단어 번호를 출력으로 얻어야 합니다. CBoW 네트워크의 구조는 다음과 같습니다:

* 입력 단어는 임베딩 레이어를 통해 전달됩니다. 이 임베딩 레이어는 우리의 Word2Vec 임베딩이 될 것이며, 따라서 이를 `embedder` 변수로 별도로 정의합니다. 이 예제에서는 임베딩 크기를 30으로 설정하지만, 더 높은 차원으로 실험해볼 수도 있습니다 (실제 Word2Vec은 300 차원입니다).
* 임베딩 벡터는 출력 단어를 예측하는 선형 레이어로 전달됩니다. 따라서 이 레이어는 `vocab_size` 개의 뉴런을 가집니다.

출력의 경우, 손실 함수로 `CrossEntropyLoss`를 사용한다면, 원핫 인코딩 없이 단어 번호만 예상 결과로 제공해야 합니다.


In [ ]:
vocab_size = len(vocab)

embedder = torch.nn.Embedding(num_embeddings = vocab_size, embedding_dim = 30)
model = torch.nn.Sequential(
    embedder,
    torch.nn.Linear(in_features = 30, out_features = vocab_size),
)

print(model)

Sequential(
  (0): Embedding(5002, 30)
  (1): Linear(in_features=30, out_features=5002, bias=True)
)


## 훈련 데이터 준비하기

이제 텍스트에서 CBoW 단어 쌍을 계산하는 주요 함수를 작성해 봅시다. 이 함수는 윈도우 크기를 지정할 수 있게 해주며, 입력 단어와 출력 단어 쌍의 집합을 반환합니다. 이 함수는 단어뿐만 아니라 벡터나 텐서에도 사용할 수 있습니다. 이를 통해 텍스트를 인코딩한 후 `to_cbow` 함수에 전달할 수 있습니다.


In [ ]:
def to_cbow(sent,window_size=2):
    res = []
    for i,x in enumerate(sent):
        for j in range(max(0,i-window_size),min(i+window_size+1,len(sent))):
            if i!=j:
                res.append([sent[j],x])
    return res

print(to_cbow(['I','like','to','train','networks']))
print(to_cbow(encode('I like to train networks', vocab)))

[['like', 'I'], ['to', 'I'], ['I', 'like'], ['to', 'like'], ['train', 'like'], ['I', 'to'], ['like', 'to'], ['train', 'to'], ['networks', 'to'], ['like', 'train'], ['to', 'train'], ['networks', 'train'], ['to', 'networks'], ['train', 'networks']]
[[232, 172], [5, 172], [172, 232], [5, 232], [0, 232], [172, 5], [232, 5], [0, 5], [1202, 5], [232, 0], [5, 0], [1202, 0], [5, 1202], [0, 1202]]


훈련 데이터셋을 준비합시다. 모든 뉴스를 살펴보고 `to_cbow`를 호출하여 단어 쌍 목록을 얻은 다음 해당 쌍을 `X`와 `Y`에 추가할 것입니다. 시간 절약을 위해 처음 10k 뉴스 항목만 고려할 것입니다 - 더 많은 시간을 기다릴 수 있고 더 나은 임베딩을 원한다면 이 제한을 쉽게 제거할 수 있습니다 :)


In [ ]:
X = []
Y = []
for i, x in zip(range(10000), train_dataset):
    for w1, w2 in to_cbow(encode(x[1], vocab), window_size = 5):
        X.append(w1)
        Y.append(w2)

X = torch.tensor(X)
Y = torch.tensor(Y)

우리는 또한 그 데이터를 하나의 데이터셋으로 변환하고, 데이터로더를 생성할 것입니다.


In [ ]:
class SimpleIterableDataset(torch.utils.data.IterableDataset):
    def __init__(self, X, Y):
        super(SimpleIterableDataset).__init__()
        self.data = []
        for i in range(len(X)):
            self.data.append( (Y[i], X[i]) )
        random.shuffle(self.data)

    def __iter__(self):
        return iter(self.data)

우리는 또한 그 데이터를 하나의 데이터셋으로 변환하고, 데이터로더를 생성할 것입니다.


In [ ]:
ds = SimpleIterableDataset(X, Y)
dl = torch.utils.data.DataLoader(ds, batch_size = 256)

이제 실제 훈련을 시작합시다. 우리는 비교적 높은 학습률을 가진 `SGD` 옵티마이저를 사용할 것입니다. 또한 `Adam`과 같은 다른 옵티마이저를 사용해보는 것도 가능합니다. 처음에는 10 에포크 동안 훈련을 진행할 것이며, 더 낮은 손실을 원한다면 이 셀을 다시 실행할 수 있습니다.


In [ ]:
def train_epoch(net, dataloader, lr = 0.01, optimizer = None, loss_fn = torch.nn.CrossEntropyLoss(), epochs = None, report_freq = 1):
    optimizer = optimizer or torch.optim.Adam(net.parameters(), lr = lr)
    loss_fn = loss_fn.to(device)
    net.train()

    for i in range(epochs):
        total_loss, j = 0, 0, 
        for labels, features in dataloader:
            optimizer.zero_grad()
            features, labels = features.to(device), labels.to(device)
            out = net(features)
            loss = loss_fn(out, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss
            j += 1
        if i % report_freq == 0:
            print(f"Epoch: {i+1}: loss={total_loss.item()/j}")

    return total_loss.item()/j

In [ ]:
train_epoch(net = model, dataloader = dl, optimizer = torch.optim.SGD(model.parameters(), lr = 0.1), loss_fn = torch.nn.CrossEntropyLoss(), epochs = 10)

Epoch: 1: loss=5.664632366860172
Epoch: 2: loss=5.632101973960962
Epoch: 3: loss=5.610399051405015
Epoch: 4: loss=5.594621561080262
Epoch: 5: loss=5.582538017415446
Epoch: 6: loss=5.572900234519603
Epoch: 7: loss=5.564951676341915
Epoch: 8: loss=5.558288112064614
Epoch: 9: loss=5.552576955031129
Epoch: 10: loss=5.547634165194347


5.547634165194347

## Word2Vec 사용해보기

Word2Vec을 사용하려면, 우리의 어휘에 있는 모든 단어에 해당하는 벡터를 추출해봅시다:


In [ ]:
vectors = torch.stack([embedder(torch.tensor(vocab[s])) for s in vocab.itos], 0)

예를 들어, 단어 **Paris**가 벡터로 어떻게 인코딩되는지 살펴보겠습니다:


In [ ]:
paris_vec = embedder(torch.tensor(vocab['paris']))
print(paris_vec)

tensor([-0.0915,  2.1224, -0.0281, -0.6819,  1.1219,  0.6458, -1.3704, -1.3314,
        -1.1437,  0.4496,  0.2301, -0.3515, -0.8485,  1.0481,  0.4386, -0.8949,
         0.5644,  1.0939, -2.5096,  3.2949, -0.2601, -0.8640,  0.1421, -0.0804,
        -0.5083, -1.0560,  0.9753, -0.5949, -1.6046,  0.5774],
       grad_fn=<EmbeddingBackward>)


Word2Vec을 사용하여 동의어를 찾는 것은 흥미롭습니다. 다음 함수는 주어진 입력에 대해 가장 가까운 `n`개의 단어를 반환합니다. 이를 찾기 위해, 우리는 $|w_i - v|$의 노름을 계산합니다. 여기서 $v$는 입력 단어에 해당하는 벡터이고, $w_i$는 어휘에서 $i$번째 단어의 인코딩입니다. 그런 다음 배열을 정렬하고 `argsort`를 사용하여 해당 인덱스를 반환하며, 목록의 첫 번째 `n` 요소를 가져옵니다. 이는 어휘에서 가장 가까운 단어의 위치를 인코딩합니다.


In [ ]:
def close_words(x, n = 5):
  vec = embedder(torch.tensor(vocab[x]))
  top5 = np.linalg.norm(vectors.detach().numpy() - vec.detach().numpy(), axis = 1).argsort()[:n]
  return [ vocab.itos[x] for x in top5 ]

close_words('microsoft')

['microsoft', 'quoted', 'lp', 'rate', 'top']

In [ ]:
close_words('basketball')

['basketball', 'lot', 'sinai', 'states', 'healthdaynews']

In [ ]:
close_words('funds')

['funds', 'travel', 'sydney', 'japan', 'business']

## 주요 내용

CBoW와 같은 기발한 기법을 사용하여 Word2Vec 모델을 훈련시킬 수 있습니다. 중심 단어를 기준으로 주변 단어를 예측하도록 훈련되는 skip-gram 모델을 직접 훈련해 보고, 그 성능이 얼마나 좋은지 확인해 보세요.



---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있지만, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전을 권위 있는 출처로 간주해야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 책임을 지지 않습니다.
